# Условие задачи:

На карте Беларуси имеется N городов, каждый с каждым связывает дорога определенной протяженности. Напишите функцию на языке “Python” нахождения ближайшего маршрута из двух городов случайно выбранных из N. 
Матрица расстояний задается случайным образом.



# Решение:
Используем алгоритм Дейкстры, учитывая, что если из города A можно попасть в город В, то и из города В можно попасть в город А. Следовательно, исходная матрица расстояний будет симметрической. Диагольньные элементы будут нулевыми, поскольку расстояние от города А до города А равно нулю.

In [1]:
import numpy as np

def createRandomMatrix( number_of_cities ):
    """ Generates symmetrical matrix of distances with random values.
        Values of diagonal elements are zeros. 
        Values of other elements are up to 650 'cause it's the width 
        of the given country. Distance can't have negative value.
    """
    matrix = np.random.randint(0, 650, size=( number_of_cities, number_of_cities )) #the matrix is generated
    symmetrical_matrix = ( matrix + matrix.T )/2                                    #make it symmetrical
    np.fill_diagonal(symmetrical_matrix, 0)                                         #fill diagonal  with zeros
    return symmetrical_matrix

Проверим работоспособность метода:

In [2]:
a = createRandomMatrix( 5 )
a

array([[  0. , 270. , 246.5, 210.5, 415. ],
       [270. ,   0. , 252. , 209.5, 421. ],
       [246.5, 252. ,   0. , 335. , 491.5],
       [210.5, 209.5, 335. ,   0. , 148.5],
       [415. , 421. , 491.5, 148.5,   0. ]])

Реализуем теперь сам алгоритм:

In [3]:
def shortestRoute( matrix_of_distances, first_city, second_city ):
    """ Using symmetrical_matrix of distances and given cities in range [1, degree_of_matrix] 
        and calculates the shortest distance from the 1st to the 2nd city.
        Returns route and total distance
    """

    first_city -= 1                                      #decrement number of city to make it from zero
    second_city -= 1

    infinity = 10 ** 10                                  #value that will be instead of actial infinity in our algorithm
    distances = [infinity] * len(matrix_of_distances)    #array of distances from first city to other cities
    distances[first_city] = 0                            #distance between first city and first city is obviously zero
    current_vertex = first_city                          #we start algorithm from the first point of the route
    minimal_distance = 0                                 #default minimal distance
    route = [i for i in range(len(matrix_of_distances))] #we will use this array to create final route
    result_route = [second_city + 1]                     #the shortest route. consists of numbers of cities in order we need to visit them to get to the second city
    visited = [False] * len(matrix_of_distances)         #this array shows whether city was visited

    while not visited[second_city]:                      #in case first city equals second city
        visited[current_vertex] = True
        if current_vertex == second_city:                #the cycle will work until we'll reach the destination
            break
        for i in range( len(matrix_of_distances) ):      #it's the actual step of algorithm. we calculate distance from one point to others
            if distances[current_vertex] + matrix_of_distances[current_vertex][i] < distances[i] and matrix_of_distances[current_vertex][i] != 0 and not visited[i] and i > 0:
                distances[i] = distances[current_vertex] + matrix_of_distances[current_vertex][i]
                route[i] = current_vertex                #here we add points to make actual route below
        tmp = []                                         #temporary array to figure out the smallest distance
        for i in range( len(distances) ):                #here we create this temporary array and don't iclude already visited items
            if not visited[i]:
                tmp.append(distances[i])
        minimal_distance = np.min(tmp)                                              #then we calculate minimal distance
        current_vertex = distances.index(minimal_distance)                          #now we find the index of the closest city
        while visited[current_vertex]:
            current_vertex = distances.index(minimal_distance, current_vertex + 1)  #here we check the value in case it has already been visited and minimal_distance equals to
                                                                                    #distance of already visited item. otherwise this while-cycle will be infinite
    vertex = second_city                      #here we reconstruct the shortest route                          
    while vertex != first_city:
        vertex = route[vertex]
        result_route.append(vertex + 1)
    result_route = result_route[::-1]
    total_distance = distances[second_city]   #from array of distances we can get the shortest distance between the first and teh second city
    return result_route, total_distance

Пример работы функции. Здесь использована не сгенерированная случайная матрица, а своя (для удобства проверки).

In [4]:
test = [[0, 7, 9, 0, 0, 14],
     [7, 0, 10, 15, 0, 0],
     [9, 10, 0, 11, 0, 2],
     [0, 15, 11, 0, 6, 0],
     [0, 0, 0, 6, 0, 9],
     [14, 0, 2, 0, 9, 0]]

shortestRoute(test, 1, 5)

([1, 3, 6, 5], 20)

Пример работы функции на сгенерированной матрице.

In [5]:
shortestRoute(a, 1, 5)

([1, 4, 5], 359.0)